In [1]:
# Check GPU
!nvidia-smi


Wed Nov  5 07:28:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Install required libraries
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors pillow


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import os

# Select model (you can change to any HF model like "stabilityai/stable-diffusion-2-1")
MODEL_NAME = "runwayml/stable-diffusion-v1-5"

# Detect GPU/CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Create output folder
os.makedirs("outputs", exist_ok=True)

# Load Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
)
pipe = pipe.to(DEVICE)


In [ ]:
prompt = "A serene landscape of mountains reflected in a crystal-clear lake, cinematic lighting, ultra-realistic, 8K resolution"

# Optional: add negative prompt
negative_prompt = "low quality, blurry, watermark, text, distorted"

# For reproducibility
seed = 123
generator = torch.Generator(device=DEVICE).manual_seed(seed)

# Generate
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    guidance_scale=7.5,
    num_inference_steps=30,
    generator=generator
).images[0]

# Save and show
output_path = f"outputs/generated_image.png"
image.save(output_path)
image.show()
print("✅ Image saved to:", output_path)


In [ ]:
prompts = [
    "A futuristic city skyline at sunset, neon reflections, ultra-detailed",
    "A cozy coffee shop interior with vintage decor, warm lighting, 4K",
    "An astronaut riding a horse on Mars, cinematic composition"
]

for i, p in enumerate(prompts):
    generator = torch.Generator(device=DEVICE).manual_seed(100 + i)
    img = pipe(p, guidance_scale=7.5, num_inference_steps=30, generator=generator).images[0]
    img.save(f"outputs/prompt_{i+1}.png")
    print(f"Saved outputs/prompt_{i+1}.png")
